In [1]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import BertForSequenceClassification, BertTokenizer

In [2]:
df = pd.read_csv("data/title_conference.csv")
df.head()

,Title,Conference
0,Innovation in Database Management: Computer Sc...,VLDB
1,High performance prime field multiplication fo...,ISCAS
2,enchanted scissors: a scissor interface for su...,SIGGRAPH
3,Detection of channel degradation attack by Int...,INFOCOM
4,Pinning a Complex Network through the Betweenn...,ISCAS


Заметим, что классы несбалансированы:

In [3]:
df.Conference.value_counts()

ISCAS       864
INFOCOM     515
VLDB        423
WWW         379
SIGGRAPH    326
Name: Conference, dtype: int64

## Encoding labels

In [4]:
possible_labels = df.Conference.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'VLDB': 0, 'ISCAS': 1, 'SIGGRAPH': 2, 'INFOCOM': 3, 'WWW': 4}

In [5]:
df["label"] = df.Conference.replace(label_dict)

## Train and Vallidation Split
Because dataset have unbalanced classes, we split the data in stratified fashion.

In [6]:
from sklearn.model_selection import train_test_split  # noqa: 402

X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=42,
    stratify=df.label.values,
)

df.data_type = ["not_set"] * df.shape[0]

df.loc[X_train, "data_type"] = "train"
df.loc[X_val, "data_type"] = "val"

df.groupby(["Conference", "label", "data_type"]).count()

C:\Users\ds13\AppData\Local\Temp\ipykernel_39904\2360066991.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.data_type = ["not_set"] * df.shape[0]


Title
Conference label data_type       
INFOCOM    3     train        438
                 val           77
ISCAS      1     train        734
                 val          130
SIGGRAPH   2     train        277
                 val           49
VLDB       0     train        359
                 val           64
WWW        4     train        322
                 val           57

## BertTokenizer

In [8]:
BERT_MODEL_TYPE = "bert-base-uncased"


tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_TYPE, do_lower_case=True)


def batch_encode_plus(data):
    return tokenizer.batch_encode_plus(
        data.Title.values,
        # Sequences will be encoded with th especial tokens relative to their model.
        add_special_tokens=True,
        # Return attention mask according to specific tokenizer.
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,  # Limit just in case.
        return_tensors="pt",  # Return pytorch compatible tensors.
    )


train_data = df[df["data_type"] == "train"]
encoded_data_train = batch_encode_plus(train_data)

val_data = df[df["data_type"] == "val"]
encoded_data_val = batch_encode_plus(val_data)

input_ids_train = encoded_data_train["input_ids"]
attention_masks_train = encoded_data_train["attention_mask"]
labels_train = torch.tensor(train_data.label.values)

input_ids_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val["attention_mask"]
labels_val = torch.tensor(val_data.label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## BERT Pre-trained model

In [9]:
model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_TYPE,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Data Loaders
Let's combine a dataset and a sampler to data loader that provides an iterable
over the given dataset.

In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler  # noqa: 402

BATCH_SIZE = 3

dataloader_train = DataLoader(
    dataset_train, sampler=RandomSampler(dataset_train), batch_size=BATCH_SIZE
)

dataloader_val = DataLoader(
    dataset_val, sampler=RandomSampler(dataset_val), batch_size=BATCH_SIZE
)

## Optimizer & Scheduler
1. We should define parameters to optimize in iterable.
2. Then specify optmizer-specific options such as epochs, learning_rate...
3. Create a schedule with a learning rate that first inreases linearly from
0 to the initial learning rate set in the optimizer (a.k.a. warm up period) and then
decreases linearly from the initial learning rate to 0.

In [11]:
from transformers import AdamW, get_linear_schedule_with_warmup  # noqa: 402

LEARNING_RATE = 1e-5
EPSILON = 1e-8
EPOCHS = 5  # Depends on dataset.

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=EPSILON)

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train) * EPOCHS
)

C:\Users\ds13\.bookmarks\Diploma@RepoAnalyzer__\repolier\.venv_win\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Performance mentrics
We will use f1 and accuracy per class.

In [14]:
import numpy as np  # noqa: 402
from sklearn.metrics import f1_score  # noqa: 402


def get_f1_score(predictions, labels):
    predictions_flattened = np.argmax(predictions, axis=1).flatten()
    labels_flattened = labels.flatten()

    return f1_score(labels_flattened, predictions_flattened, average="weighted")

In [22]:
def accuracy_per_class(predictions, labels):
    # Inverse the dictionary.
    labels_lookup_table = {v: k for k, v in label_dict.items()}

    predictions_flattened = np.argmax(predictions, axis=1).flatten()
    labels_flattened = labels.flatten()

    for label in np.unique(labels_flattened):
        y_predicted = predictions_flattened[labels_flattened == label]
        y_true = labels_flattened[labels_flattened == label]

        print(f"Class: {labels_lookup_table[label]}")
        print(f"Accuracy: {len(y_predicted[y_predicted==label])}/{len(y_true)}")

### Training loop

In [16]:
import random  # noqa: 402

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(device)

cuda


In [17]:
from pathlib import Path  # noqa: 402

Path("models").mkdir(parents=True, exist_ok=True)


def map_batch_to_inputs(batch):
    return {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "labels": batch[2],
    }


def evaluate(dataloader_val):
    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:
        batch = tuple(b.to(device) for b in batch)

        inputs = map_batch_to_inputs(batch)

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs["labels"].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total / len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals


for epoch in tqdm(range(1, EPOCHS + 1)):
    model.train()

    loss_train_total = 0

    progress_bar = tqdm(
        dataloader_train, desc="Epoch {:1d}".format(epoch), leave=False, disable=False
    )

    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = map_batch_to_inputs(batch)

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(
            {"training_loss": "{:.3f}".format(loss.item() / len(batch))}
        )

    torch.save(model.state_dict(), f"models/finetuned_BERT_epoch_{epoch}.model")

    tqdm.write(f"\nEpoch {epoch}")

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f"Training loss: {loss_train_avg}")

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = get_f1_score(predictions, true_vals)
    tqdm.write(f"Validation loss: {val_loss}")
    tqdm.write(f"F1 Score (weighted): {val_f1}")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/710 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8888539495497523
Validation loss: 0.5301494788348912
F1 Score (weighted): 0.8349282731792835


Epoch 2:   0%|          | 0/710 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.5080354237650245
Validation loss: 0.6182522111893853
F1 Score (weighted): 0.8465429330535159


...


Epoch 3
Training loss: 0.3500029952366981
Validation loss: 0.6915738689620787
F1 Score (weighted): 0.8477420006215807


Epoch 4:   0%|          | 0/710 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.23502694387300413
Validation loss: 0.8227984245265058
F1 Score (weighted): 0.8365971847893648


Epoch 5:   0%|          | 0/710 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.16323284446893088
Validation loss: 0.789898806753742
F1 Score (weighted): 0.8472826151498132


## Loading and evaluating the model

In [19]:
model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_TYPE,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [20]:
model.load_state_dict(
    torch.load("models/finetuned_BERT_epoch_1.model", map_location=torch.device("cuda"))
)

<All keys matched successfully>

In [23]:
_, predictions, true_vals = evaluate(dataloader_val)
accuracy_per_class(predictions, true_vals)

[[ 3.0668375  -1.4040668  -1.1406454  -0.40617484  0.13424563]
 [-0.20288248  0.817448    2.4179237  -1.461867   -0.2450478 ]
 [-0.09810138 -1.5999974  -1.0052575   0.30795276  2.0977244 ]
 ...
 [ 2.728063   -0.6249323   0.00463311 -1.4429905   0.21042904]
 [ 2.5265696  -1.1070975  -1.8461831   0.8716704  -0.29535842]
 [ 0.5891698  -1.5345939  -0.6063291  -1.2490883   2.9028754 ]]
[0 2 4 1 3 1 2 1 2 1 0 1 4 1 1 0 4 1 1 1 2 1 4 3 3 3 2 1 3 1 1 3 3 3 4 1 0
 0 2 1 1 1 3 3 3 4 0 2 1 1 3 3 4 1 1 3 4 2 0 0 1 3 3 3 0 0 3 1 1 1 1 0 2 1
 0 2 1 3 3 1 2 4 1 2 2 4 2 0 4 4 4 3 1 4 2 3 0 4 2 3 0 1 1 1 0 1 1 2 4 0 0
 3 3 0 1 2 3 0 4 1 0 1 1 4 0 3 3 0 2 1 1 2 1 3 1 2 1 1 0 1 1 3 3 3 0 2 2 0
 2 4 3 2 3 1 1 1 3 2 0 3 1 2 1 0 1 1 1 3 2 1 1 0 2 4 1 1 1 2 3 3 1 1 1 3 0
 0 4 2 4 3 4 4 1 3 1 1 3 3 1 2 2 1 1 3 4 1 0 0 4 4 4 1 2 4 0 4 4 3 2 0 0 0
 1 1 4 4 2 1 4 4 2 2 3 1 1 4 1 1 0 1 2 3 0 4 1 4 4 3 1 4 1 0 1 4 2 1 1 0 4
 0 4 1 3 1 1 3 3 1 1 1 0 1 1 3 0 4 1 1 4 0 1 0 4 3 1 1 0 2 1 0 1 1 4 0 1 1
 1 4 3 1 4 1 3 1